In [ ]:
import os
import sys

import numpy as np
import dolfin as dl

sys.path.append(os.path.join(os.getenv("DT4CO_PATH"), "src"))
from dt4co.experiments import ExperimentFactory
from dt4co.utils.mesh_utils import load_mesh
from dt4co.dataModel import PatientData
from dt4co.utils.fenics_io import read_mv_from_h5

sys.path.append(os.environ.get("HIPPYLIB_PATH"))
import hippylib as hp

If your program has OpenMP sections, this can cause over-subscription of cores and consequently poor performance
To avoid this, please re-run your application after setting MV2_ENABLE_AFFINITY=0
Use MV2_USE_THREAD_WARNING=0 to suppress this message


In [2]:
# MPI setup.
COMM = dl.MPI.comm_world
rank = COMM.rank
nproc = COMM.size

In [3]:
COHORT= ["W03", "W10", "W11", "W16", "W18", "W20", "W29", "W31", "W35", "W36", "W39", "W43", "W50", "W53"]
DATADIR = os.path.join(os.getenv("SCRATCH"), "hgg")

In [4]:
# outlier / non-physical parameter detection
DMAX = 1.
DMIN = 1e-3

KMAX = 0.15
KMIN = 1e-3

# based on z-score
def reject_outliers(data, m=1):
    return data[abs(data - np.mean(data)) < m * np.std(data)]

In [5]:
RD_DVALUES = []
RD_KVALUES = []

for idx, SUBID in enumerate(COHORT):
    print(f"Processing Patient: {SUBID}")
    
    MESH_FPATH = os.path.join(DATADIR, f"sub-{SUBID}", "bip_mesh", "bip_mesh.xdmf")
    PATIENT_DIR = os.path.join(os.getenv("WORK"), "data/IvyGAP/", SUBID)
    PATIENT_INFO = os.path.join(PATIENT_DIR, "patient_info.json")
    
    pinfo = PatientData(PATIENT_INFO, PATIENT_DIR)
    
    # Experiment setup.
    factory = ExperimentFactory(pinfo)
    exp = factory.get_experiment('rd')

    mesh = load_mesh(COMM, MESH_FPATH)
    Vh = exp.setupFunctionSpaces(mesh, mle=True)

    # read back the map point.
    SAMPLE_FPATH = os.path.join(DATADIR, f"sub-{SUBID}", "dmrd_mle", "dmrd_mle_param_data.h5")
    # SAMPLE_FPATH = os.path.join(DATADIR, f"sub-{SUBID}", "dmrdtx_mle", "dmrdtx_mle_param_data.h5")
    DATA_NAME = "map"
    
    template_fun = dl.Function(Vh[hp.PARAMETER])
    
    mmv = hp.MultiVector(template_fun.vector(), 1)
    try:
        read_mv_from_h5(COMM, mmv, Vh[hp.PARAMETER], SAMPLE_FPATH, name=[DATA_NAME])
    
        # report.
        dval, kval = mmv[0].vec().getArray()
        print(f"MLE log diffusion: {dval}")
        print(f"MLE log proliferation: {kval}")
        print(f"MLE diffusion: {np.exp(dval)}")
        print(f"MLE proliferation: {np.exp(kval)}")
    
        RD_DVALUES.append(dval)
        RD_KVALUES.append(kval)
    except:
        print("Can't process patient")

Processing Patient: W03
MLE log diffusion: 0.1143035003612636
MLE log proliferation: -21.922898628312048
MLE diffusion: 1.1210923251817877
MLE proliferation: 3.0130483333676464e-10
Processing Patient: W10
MLE log diffusion: -1.1449908089568364
MLE log proliferation: -5.3926840706878
MLE diffusion: 0.3182268426136243
MLE proliferation: 0.00454974509489878
Processing Patient: W11
MLE log diffusion: -1.706727316271854
MLE log proliferation: -22.660688031873754
MLE diffusion: 0.1814586787888884
MLE proliferation: 1.4407486842098227e-10
Processing Patient: W16
MLE log diffusion: -0.2031689260893507
MLE log proliferation: -5.194750749761688
MLE diffusion: 0.8161403623804757
MLE proliferation: 0.0055455983844632465
Processing Patient: W18
MLE log diffusion: 2.061072280786489
MLE log proliferation: -22.081476239067886
MLE diffusion: 7.854387405203522
MLE proliferation: 2.571205077794512e-10
Processing Patient: W20
Can't process patient
Processing Patient: W29
MLE log diffusion: -1.299295201065

In [6]:
print("Reaction diffusion WITHOUT treatment")
DVALUES = RD_DVALUES
KVALUES = RD_KVALUES

phys_d_vals = np.exp(DVALUES)
phys_d_vals = phys_d_vals[phys_d_vals > DMIN]
phys_d_vals = phys_d_vals[phys_d_vals < DMAX]

phys_k_vals = np.exp(KVALUES)
phys_k_vals = phys_k_vals[phys_k_vals < KMAX]
phys_k_vals = phys_k_vals[phys_k_vals > KMIN]

print(f"Number of diffusion values: {len(phys_d_vals)}")
print(f"Number of proliferation values: {len(phys_k_vals)}")

print(f"Mean diffusion is: {np.mean(phys_d_vals)}")
print(f"Variance of diffusion is: {np.var(phys_d_vals)}")

print(f"Mean proliferation rate is: {np.mean(phys_k_vals)}")
print(f"Variance of proliferation rate is: {np.var(phys_k_vals)}")

log_d_vals = np.log(phys_d_vals)
log_k_vals = np.log(phys_k_vals)

print("###")
print(f"Log mean diffusion is: {np.mean(log_d_vals)}")
print(f"Log variance of diffusion is: {np.var(log_d_vals)}")

print(f"Log mean proliferation rate is: {np.mean(log_k_vals)}")
print(f"Log variance of proliferation rate is: {np.var(log_k_vals)}")

Reaction diffusion WITHOUT treatment
Number of diffusion values: 7
Number of proliferation values: 3
Mean diffusion is: 0.3959213192772872
Variance of diffusion is: 0.051458666473056476
Mean proliferation rate is: 0.0054595610785671544
Variance of proliferation rate is: 5.045929506988528e-07
###
Log mean diffusion is: -1.1063448588811264
Log variance of diffusion is: 0.3853095522796434
Log mean proliferation rate is: -5.219096150659049
Log variance of proliferation rate is: 0.017666264680481242


In [7]:
RDTX_DVALUES = []
RDTX_KVALUES = []

for idx, SUBID in enumerate(COHORT):
    print(f"Processing Patient: {SUBID}")
    
    MESH_FPATH = os.path.join(DATADIR, f"sub-{SUBID}", "bip_mesh", "bip_mesh.xdmf")
    PATIENT_DIR = os.path.join(os.getenv("WORK"), "data/IvyGAP/", SUBID)
    PATIENT_INFO = os.path.join(PATIENT_DIR, "patient_info.json")
    
    pinfo = PatientData(PATIENT_INFO, PATIENT_DIR)
    
    # Experiment setup.
    factory = ExperimentFactory(pinfo)
    exp = factory.get_experiment('rdtx')

    mesh = load_mesh(COMM, MESH_FPATH)
    Vh = exp.setupFunctionSpaces(mesh, mle=True)

    # read back the map point.
    SAMPLE_FPATH = os.path.join(DATADIR, f"sub-{SUBID}", "dmrdtx_mle", "dmrdtx_mle_param_data.h5")
    DATA_NAME = "map"
    
    template_fun = dl.Function(Vh[hp.PARAMETER])
    
    mmv = hp.MultiVector(template_fun.vector(), 1)
    try:
        read_mv_from_h5(COMM, mmv, Vh[hp.PARAMETER], SAMPLE_FPATH, name=[DATA_NAME])
    
        # report.
        dval, kval = mmv[0].vec().getArray()
        print(f"MLE log diffusion: {dval}")
        print(f"MLE log proliferation: {kval}")
        print(f"MLE diffusion: {np.exp(dval)}")
        print(f"MLE proliferation: {np.exp(kval)}")
    
        RDTX_DVALUES.append(dval)
        RDTX_KVALUES.append(kval)
    except:
        print("Can't process patient")

Processing Patient: W03
MLE log diffusion: -2.443081478669667
MLE log proliferation: -2.817342700006679
MLE diffusion: 0.08689268055104375
MLE proliferation: 0.05976454421266214
Processing Patient: W10
MLE log diffusion: -49.82932868352395
MLE log proliferation: -979.8084185123718
MLE diffusion: 2.287691801596685e-22
MLE proliferation: 0.0
Processing Patient: W11
MLE log diffusion: -3.585086338601014
MLE log proliferation: -3.164030796146209
MLE diffusion: 0.02773427298847698
MLE proliferation: 0.04225507575650025
Processing Patient: W16
MLE log diffusion: -3.0878809202027644
MLE log proliferation: -1.346505750933838
MLE diffusion: 0.045598478912382036
MLE proliferation: 0.2601476951623734
Processing Patient: W18
MLE log diffusion: -2.302602314143742
MLE log proliferation: -10.664932068932993
MLE diffusion: 0.09999827789985868
MLE proliferation: 2.3349568143790975e-05
Processing Patient: W20
Can't process patient
Processing Patient: W29
MLE log diffusion: -2.36765873377681
MLE log prol

In [8]:
print("Reaction diffusion WITH treatment")

DVALUES = RDTX_DVALUES
KVALUES = RDTX_KVALUES

phys_d_vals = np.exp(DVALUES)
phys_d_vals = phys_d_vals[phys_d_vals > DMIN]
phys_d_vals = phys_d_vals[phys_d_vals < DMAX]

phys_k_vals = np.exp(KVALUES)
phys_k_vals = phys_k_vals[phys_k_vals < KMAX]
phys_k_vals = phys_k_vals[phys_k_vals > KMIN]

phys_d_vals = reject_outliers(phys_d_vals)
phys_k_vals = reject_outliers(phys_k_vals)

print(f"Number of diffusion values: {len(phys_d_vals)}")
print(f"Number of proliferation values: {len(phys_k_vals)}")

print(f"Mean diffusion is: {np.mean(phys_d_vals)}")
print(f"Variance of diffusion is: {np.var(phys_d_vals)}")

print(f"Mean proliferation rate is: {np.mean(phys_k_vals)}")
print(f"Variance of proliferation rate is: {np.var(phys_k_vals)}")

log_d_vals = np.log(phys_d_vals)
log_k_vals = np.log(phys_k_vals)

print("###")
print(f"Log mean diffusion is: {np.mean(log_d_vals)}")
print(f"Log variance of diffusion is: {np.var(log_d_vals)}")

print(f"Log mean proliferation rate is: {np.mean(log_k_vals)}")
print(f"Log variance of proliferation rate is: {np.var(log_k_vals)}")

Reaction diffusion WITH treatment
Number of diffusion values: 4
Number of proliferation values: 6
Mean diffusion is: 0.06808730917391463
Variance of diffusion is: 0.0004990689941995712
Mean proliferation rate is: 0.058786887312270775
Variance of proliferation rate is: 0.00013186234064586555
###
Log mean diffusion is: -2.7435752768386137
Log variance of diffusion is: 0.11511241912075106
Log mean proliferation rate is: -2.853376048422435
Log variance of proliferation rate is: 0.03960178345019936


In [12]:
import scipy
def logvar2physvar(physmu, logvar):
    """Convert log-variance to physical variance.
    """
    f = lambda x : (np.exp(x) - 1)*np.exp(2*physmu + x) - logvar

    physvar = scipy.optimize.fsolve(f, 0.1)
    
    return physvar

In [15]:
phys_d_var = logvar2physvar(np.mean(phys_d_vals), np.var(log_d_vals))
phys_k_var = logvar2physvar(np.mean(phys_k_vals), np.var(log_k_vals))

print(f"Physical variance of diffusion is: {phys_d_var}")
print(f"Physical variance of proliferation is: {phys_k_var}")

Physical variance of diffusion is: [0.08800584]
Physical variance of proliferation is: [0.03348272]
